# Correlation and feature importance

Ok, after looking at the general trends on the data, as well as lookin at proof that different subgroups have very different behaviors, it is time to figure out how variables correlate with each other, as well as finding which are the variables that better describe the variable we are interested in.

In this case, climbers will probably want to learn which variables correlate with their climbing performance, so we will use some analytic tools to find which variables are the best at predicting climbing performance. And being good at predicting something, usually means that modifying that would modify the variable of interest as you desire (become better climbers in this example).

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
fullData = pd.read_csv("filteredData.csv")
fullData.describe()

,FEMALE,AGE,YRS_CLIMBING,START_AGE,PARENTS,HEIGHT,APEINDEX,WEIGHT,BMI,OCC_STUDY,...,B_INPOTENTIAL,B_OUTPOTENTIAL,S_FLASH,S_REDP,S_FLASHCONF,S_FINISHCONF,S_AVG,S_FPOTENTIAL,S_MPOTENTIAL,S_INCONSISTENCY
count,155.000000,155.00000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,...,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000
mean,0.400000,27.63871,5.356774,22.281935,0.083871,171.728258,0.696774,63.649677,21.493991,0.438710,...,1.335484,1.200000,4.922581,6.993548,3.496774,6.025806,5.359677,2.070968,-0.967742,1.425806
std,0.491486,8.26971,5.490145,6.645587,0.278093,9.735438,2.974729,10.270835,2.257924,0.497838,...,0.948529,1.164407,3.853714,4.614343,3.187586,4.496678,3.905341,1.844999,1.828417,1.357750
min,0.000000,15.00000,0.500000,10.000000,0.000000,150.000000,-10.000000,40.000000,16.003658,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.000000,0.000000
25%,0.000000,22.00000,2.000000,19.000000,0.000000,164.500000,0.000000,56.000000,19.993063,0.000000,...,1.000000,0.000000,1.000000,3.000000,0.500000,2.000000,2.000000,0.000000,-2.000000,0.000000
50%,0.000000,25.00000,4.000000,21.000000,0.000000,170.000000,0.000000,63.000000,21.258503,0.000000,...,1.000000,1.000000,5.000000,8.000000,3.000000,6.000000,5.500000,2.000000,-1.000000,1.000000
75%,1.000000,30.50000,6.000000,24.250000,0.000000,179.000000,0.000000,70.000000,22.810340,1.000000,...,2.000000,2.000000,8.000000,10.000000,6.000000,9.000000,8.250000,3.000000,0.000000,3.000000
max,1.000000,58.00000,31.000000,53.000000,1.000000,208.000000,13.000000,108.000000,27.777778,1.000000,...,6.000000,7.000000,15.000000,18.000000,12.000000,17.000000,15.250000,8.000000,3.000000,5.000000


In [3]:
normData = pd.read_csv("normData.csv")
normData.describe()

,FEMALE,AGE,YRS_CLIMBING,START_AGE,PARENTS,HEIGHT,APEINDEX,WEIGHT,BMI,OCC_STUDY,...,B_INPOTENTIAL,B_OUTPOTENTIAL,S_FLASH,S_REDP,S_FLASHCONF,S_FINISHCONF,S_AVG,S_FPOTENTIAL,S_MPOTENTIAL,S_INCONSISTENCY
count,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,...,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000
mean,0.400000,0.293923,0.159238,0.285626,0.083871,0.374625,0.465077,0.347789,0.466305,0.438710,...,0.222581,0.171429,0.328172,0.388530,0.291398,0.354459,0.351454,0.258871,0.504032,0.285161
std,0.491486,0.192319,0.180005,0.154549,0.278093,0.167852,0.129336,0.151042,0.191770,0.497838,...,0.158088,0.166344,0.256914,0.256352,0.265632,0.264510,0.256088,0.230625,0.228552,0.271550
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.162791,0.049180,0.209302,0.000000,0.250000,0.434783,0.235294,0.338828,0.000000,...,0.166667,0.000000,0.066667,0.166667,0.041667,0.117647,0.131148,0.000000,0.375000,0.000000
50%,0.000000,0.232558,0.114754,0.255814,0.000000,0.344828,0.434783,0.338235,0.446305,0.000000,...,0.166667,0.142857,0.333333,0.444444,0.250000,0.352941,0.360656,0.250000,0.500000,0.200000
75%,1.000000,0.360465,0.180328,0.331395,0.000000,0.500000,0.434783,0.441176,0.578105,1.000000,...,0.333333,0.285714,0.533333,0.555556,0.500000,0.529412,0.540984,0.375000,0.625000,0.600000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
data = normData.copy()
data.iloc[:,-17:] = fullData.iloc[:,-17:]
data.describe()

,FEMALE,AGE,YRS_CLIMBING,START_AGE,PARENTS,HEIGHT,APEINDEX,WEIGHT,BMI,OCC_STUDY,...,B_INPOTENTIAL,B_OUTPOTENTIAL,S_FLASH,S_REDP,S_FLASHCONF,S_FINISHCONF,S_AVG,S_FPOTENTIAL,S_MPOTENTIAL,S_INCONSISTENCY
count,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,...,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000
mean,0.400000,0.293923,0.159238,0.285626,0.083871,0.374625,0.465077,0.347789,0.466305,0.438710,...,1.335484,1.200000,4.922581,6.993548,3.496774,6.025806,5.359677,2.070968,-0.967742,1.425806
std,0.491486,0.192319,0.180005,0.154549,0.278093,0.167852,0.129336,0.151042,0.191770,0.497838,...,0.948529,1.164407,3.853714,4.614343,3.187586,4.496678,3.905341,1.844999,1.828417,1.357750
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.000000,0.000000
25%,0.000000,0.162791,0.049180,0.209302,0.000000,0.250000,0.434783,0.235294,0.338828,0.000000,...,1.000000,0.000000,1.000000,3.000000,0.500000,2.000000,2.000000,0.000000,-2.000000,0.000000
50%,0.000000,0.232558,0.114754,0.255814,0.000000,0.344828,0.434783,0.338235,0.446305,0.000000,...,1.000000,1.000000,5.000000,8.000000,3.000000,6.000000,5.500000,2.000000,-1.000000,1.000000
75%,1.000000,0.360465,0.180328,0.331395,0.000000,0.500000,0.434783,0.441176,0.578105,1.000000,...,2.000000,2.000000,8.000000,10.000000,6.000000,9.000000,8.250000,3.000000,0.000000,3.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,6.000000,7.000000,15.000000,18.000000,12.000000,17.000000,15.250000,8.000000,3.000000,5.000000
